# 📝 Exercise M1.04

The goal of this exercise is to evaluate the impact of using an arbitrary
integer encoding for categorical variables along with a linear classification
model such as Logistic Regression.

To do so, let's try to use `OrdinalEncoder` to preprocess the categorical
variables. This preprocessor is assembled in a pipeline with
`LogisticRegression`. The generalization performance of the pipeline can be
evaluated by cross-validation and then compared to the score obtained when
using `OneHotEncoder` or to some other baseline score.

First, we load the dataset.

In [1]:
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

In [2]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

In the previous notebook, we used `sklearn.compose.make_column_selector` to
automatically select columns with a specific data type (also called `dtype`).
Here, we use this selector to get only the columns containing strings (column
with `object` dtype) that correspond to categorical features in our dataset.

In [3]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
data_categorical = data[categorical_columns]

Define a scikit-learn pipeline composed of an `OrdinalEncoder` and a
`LogisticRegression` classifier.

Because `OrdinalEncoder` can raise errors if it sees an unknown category at
prediction time, you can set the `handle_unknown="use_encoded_value"` and
`unknown_value` parameters. You can refer to the [scikit-learn
documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)
for more details regarding these parameters.

<div class="alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">My Note</p>
<p>
OrdinalEncoder will replace unknown values with np.nan. This will lead to an error in the LogisticRegression. We can use <a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html">KNNImputer</a> to impute missing values.
</p>
</div>

In [29]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.impute import KNNImputer


# Write your code here.
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan).set_output(transform="pandas")

model = make_pipeline(encoder, KNNImputer(), LogisticRegression(max_iter=500))

Your model is now defined. Evaluate it using a cross-validation using
`sklearn.model_selection.cross_validate`.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Be aware that if an error happened during the cross-validation,
<tt class="docutils literal">cross_validate</tt> would raise a warning and return NaN (Not a Number) as scores.
To make it raise a standard Python exception with a traceback, you can pass
the <tt class="docutils literal"><span class="pre">error_score="raise"</span></tt> argument in the call to <tt class="docutils literal">cross_validate</tt>. An
exception would be raised instead of a warning at the first encountered problem
and <tt class="docutils literal">cross_validate</tt> would stop right away instead of returning NaN values.
This is particularly handy when developing complex machine learning pipelines.</p>
</div>

In [36]:
from sklearn.model_selection import cross_validate

# Write your code here.
cv_results = cross_validate(model, data_categorical, target)
for key, values in cv_results.items():
    print(f"{key+':':<15} {values}")
scores = cv_results["test_score"]
print(f"The accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")

fit_time:       [0.44135189 0.27168274 0.36635756 0.38192868 0.32005858]
score_time:     [0.03015709 0.03336787 0.03326845 0.03409624 0.02983809]
test_score:     [0.75514382 0.75555328 0.75573301 0.75307125 0.75788288]
The accuracy is: 0.755 ± 0.002


Now, we would like to compare the generalization performance of our previous
model with a new model where instead of using an `OrdinalEncoder`, we use a
`OneHotEncoder`. Repeat the model evaluation using cross-validation. Compare
the score of both models and conclude on the impact of choosing a specific
encoding strategy when using a linear model.

In [37]:
from sklearn.preprocessing import OneHotEncoder

# Write your code here.
encoder = OneHotEncoder(handle_unknown='ignore')
model = make_pipeline(encoder, LogisticRegression(max_iter=500))
cv_results = cross_validate(model, data_categorical, target)
for key, values in cv_results.items():
    print(f"{key+':':<15} {values}")
scores = cv_results["test_score"]
print(f"The accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")

fit_time:       [0.70531893 0.71347737 0.632792   0.71288228 0.6841476 ]
score_time:     [0.02020502 0.01551723 0.03628802 0.03337145 0.03082418]
test_score:     [0.83222438 0.83560242 0.82872645 0.83312858 0.83466421]
The accuracy is: 0.833 ± 0.002
